<a href="https://colab.research.google.com/github/bredecker/Mapping_Earthquakes/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,075 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,324 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http://secu

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-12 22:53:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.48MB/s    in 0.2s    

2023-04-12 22:53:12 (5.48 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
# Load in the review data from S3 into the dataframe
url = "https://unbearable-1-project-bucket.s3.us-east-2.amazonaws.com/Diabetes_Key_Indicators.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+---+------+------+-----------+------+-----+-------+--------+-------------+-------------+--------+--------+--------+--------+--------+-------+-------+--------+-------+--------------------+--------+--------+--------+--------+--------+-------+-------+--------+--------+-------+--------+-------+------+-------+--------+--------+--------+--------+--------+--------+--------+--------+-------+------+--------+-------+--------+--------+--------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+-------+-----+--------+--------+--------+--------+--------+-------+--------+-------+--------+-------+-------+----+-----+------+--------+--------+--------+--------+--------+-------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+-------+----

In [5]:
new_df = df.select(["DIABETE4",
                    "_BMI5",
                    "SMOKE100",
                    "DRNK3GE5",
                    "EDUCA",
                    "GENHLTH",
                    "_AGE80",
                    "EXERANY2",
                    "FRUIT2",
                    "VEGETAB2",
                    "INCOME3",
                    "MEDCOST1",
                    "_HISPANC",
                    "_MRACE1",
                    "_PRACE1",
                    "_SEX"])
new_df.show()

+--------+------+--------+--------+-----+-------+------+--------+------+--------+-------+--------+--------+-------+-------+----+
|DIABETE4| _BMI5|SMOKE100|DRNK3GE5|EDUCA|GENHLTH|_AGE80|EXERANY2|FRUIT2|VEGETAB2|INCOME3|MEDCOST1|_HISPANC|_MRACE1|_PRACE1|_SEX|
+--------+------+--------+--------+-----+-------+------+--------+------+--------+-------+--------+--------+-------+-------+----+
|     3.0|1454.0|     1.0|    null|  4.0|    5.0|  70.0|     2.0| 101.0|   101.0|    5.0|     2.0|     2.0|    1.0|    1.0| 2.0|
|     1.0|  null|     2.0|    null|  6.0|    3.0|  67.0|     1.0| 101.0|   207.0|   77.0|     2.0|     2.0|    2.0|    2.0| 2.0|
|     1.0|2829.0|     2.0|    null|  4.0|    2.0|  72.0|     2.0| 101.0|   203.0|    3.0|     2.0|     2.0|    2.0|    2.0| 2.0|
|     1.0|3347.0|     2.0|     1.0|  4.0|    2.0|  62.0|     1.0| 203.0|   205.0|    7.0|     2.0|     2.0|    1.0|    1.0| 2.0|
|     1.0|2873.0|     2.0|    null|  3.0|    5.0|  76.0|     1.0| 101.0|   101.0|    4.0|     2.0

### Clean up Columns

In [6]:
# Ran command to remove NaN values
import pandas as pd
temp_df = new_df.toPandas()
pandas_df = temp_df.dropna()

pandas_df

,DIABETE4,_BMI5,SMOKE100,DRNK3GE5,EDUCA,GENHLTH,_AGE80,EXERANY2,FRUIT2,VEGETAB2,INCOME3,MEDCOST1,_HISPANC,_MRACE1,_PRACE1,_SEX
3,1.0,3347.0,2.0,1.0,4.0,2.0,62.0,1.0,203.0,205.0,7.0,2.0,2.0,1.0,1.0,2.0
9,3.0,3994.0,1.0,88.0,4.0,3.0,65.0,1.0,101.0,204.0,8.0,2.0,2.0,1.0,1.0,2.0
19,3.0,2986.0,2.0,88.0,4.0,2.0,53.0,2.0,202.0,207.0,6.0,2.0,2.0,1.0,1.0,2.0
22,3.0,3587.0,1.0,88.0,4.0,4.0,68.0,2.0,302.0,306.0,8.0,2.0,2.0,1.0,1.0,1.0
24,3.0,2092.0,1.0,88.0,4.0,5.0,80.0,1.0,101.0,204.0,1.0,2.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438687,1.0,2148.0,2.0,8.0,2.0,4.0,66.0,1.0,315.0,205.0,3.0,2.0,1.0,2.0,2.0,1.0
438688,2.0,2469.0,1.0,88.0,4.0,2.0,30.0,1.0,210.0,103.0,5.0,1.0,2.0,99.0,99.0,2.0
438690,3.0,3068.0,2.0,88.0,6.0,2.0,54.0,1.0,102.0,101.0,10.0,2.0,2.0,2.0,2.0,1.0
438691,3.0,2373.0,2.0,88.0,4.0,2.0,67.0,1.0,101.0,101.0,6.0,2.0,2.0,99.0,99.0,1.0


In [7]:
# check for unique values to replace for encoding
pandas_df.EDUCA.unique()

array([4., 5., 6., 3., 2., 1., 9.])

In [19]:
pandas_df = pandas_df.replace({"EDUCA":{4.:"High School Grad"}})
pandas_df = pandas_df.replace({"EDUCA":{6.:"College Grad"}})
pandas_df = pandas_df.replace({"EDUCA":{1.:"Did Not Attend"}})
pandas_df = pandas_df.replace({"EDUCA":{2.:"Elementary"}})
pandas_df = pandas_df.replace({"EDUCA":{3.:"Some High School"}})
pandas_df = pandas_df.replace({"EDUCA":{5.:"Some College/Tech School"}})
pandas_df = pandas_df.replace({"EDUCA":{9.:"NaN"}})

In [20]:
# (Completed in Clean Up Columns Step) Also, run the command to remove NaN values as we can't use them, steps are here https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# do replace values for EDUCA, GENHLTH, DRNK3GE5, EXERANY2, MEDCOST1, _SEX, INCOME3, SMOKE100
# If responses are dont know or refused to answer those responses will need to be excluded as they dont tell us anything

pandas_df.reset_index()

,index,DIABETE4,_BMI5,SMOKE100,DRNK3GE5,EDUCA,GENHLTH,_AGE80,EXERANY2,FRUIT2,VEGETAB2,INCOME3,MEDCOST1,_HISPANC,_MRACE1,_PRACE1,_SEX
0,3,1.0,3347.0,2.0,1.0,High School Grad,2.0,62.0,1.0,203.0,205.0,7.0,2.0,2.0,1.0,1.0,2.0
1,9,3.0,3994.0,1.0,88.0,High School Grad,3.0,65.0,1.0,101.0,204.0,8.0,2.0,2.0,1.0,1.0,2.0
2,19,3.0,2986.0,2.0,88.0,High School Grad,2.0,53.0,2.0,202.0,207.0,6.0,2.0,2.0,1.0,1.0,2.0
3,22,3.0,3587.0,1.0,88.0,High School Grad,4.0,68.0,2.0,302.0,306.0,8.0,2.0,2.0,1.0,1.0,1.0
4,24,3.0,2092.0,1.0,88.0,High School Grad,5.0,80.0,1.0,101.0,204.0,1.0,2.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191841,438687,1.0,2148.0,2.0,8.0,Elementary,4.0,66.0,1.0,315.0,205.0,3.0,2.0,1.0,2.0,2.0,1.0
191842,438688,2.0,2469.0,1.0,88.0,High School Grad,2.0,30.0,1.0,210.0,103.0,5.0,1.0,2.0,99.0,99.0,2.0
191843,438690,3.0,3068.0,2.0,88.0,College Grad,2.0,54.0,1.0,102.0,101.0,10.0,2.0,2.0,2.0,2.0,1.0
191844,438691,3.0,2373.0,2.0,88.0,High School Grad,2.0,67.0,1.0,101.0,101.0,6.0,2.0,2.0,99.0,99.0,1.0


In [30]:
# Check sum of null values remaining
pandas_df.isna().sum()




DIABETE4    0
_BMI5       0
SMOKE100    0
DRNK3GE5    0
EDUCA       0
GENHLTH     0
_AGE80      0
EXERANY2    0
FRUIT2      0
VEGETAB2    0
INCOME3     0
MEDCOST1    0
_HISPANC    0
_MRACE1     0
_PRACE1     0
_SEX        0
dtype: int64

In [ ]:
# Remove answers of "don't know or refused to answer" for EDUCA column